##### Copyright 2019 The TensorFlow Authors.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

# from nltk import ngrams
# from collections import Counter
import  nltk.translate.bleu_score as bleu

## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/. This dataset contains language translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

There are a variety of languages available, but we'll use the English-Spanish dataset. For convenience, we've hosted a copy of this dataset on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [3]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

In [4]:
# Converts the unicode file to ascii
# def unicode_to_ascii(s):
#   return ''.join(c for c in unicodedata.normalize('NFD', s)
#       if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
#   w = unicode_to_ascii(w.lower().strip())
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [5]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [8]:
def create_dataset(examples, num1,num2):
#   lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(pt_.numpy().decode("utf-8", "ignore")), preprocess_sentence(en_.numpy().decode("utf-8", "ignore"))]  for pt_, en_ in examples]
  word_pairs = word_pairs[num1:num2]
  return zip(*word_pairs)

In [9]:
pt, en = create_dataset(train_examples,None,None)
print(pt[-1])
print(en[-1])

<start> os meus alunos t m problemas , problemas sociais , emocionais e econ micos , que voc s nem podem imaginar . <end>
<start> my students have problems social , emotional and economic problems you could never imagine . <end>


In [12]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [13]:
def load_dataset(examples,num1,num2):
  # creating cleaned input, output pairs
  inp_lang, targ_lang = create_dataset(examples,num1,num2)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [14]:
# Try experimenting with the size of that dataset
# num_examples = 30000
num1 =0
num2 = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(train_examples,num1,num2)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [15]:
input_tensor_train = input_tensor
target_tensor_train = target_tensor

print(len(input_tensor_train), len(target_tensor_train))

30000 30000


In [16]:
input_tensor_train

array([[   3,    9,   44, ...,    0,    0,    0],
       [   3,   25,    9, ...,    0,    0,    0],
       [   3,   25,   61, ...,    0,    0,    0],
       ...,
       [   3,   30,  161, ...,    0,    0,    0],
       [   3,  216,   13, ...,    0,    0,    0],
       [   3,   15, 5956, ...,    0,    0,    0]])

### Create a tf.data dataset

In [23]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 4
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 16 #100
units = 64 #1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [24]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([4, 165]), TensorShape([4, 137]))

# Encoder Model

In [25]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [26]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (4, 165, 64)
Encoder Hidden state shape: (batch size, units) (4, 64)


# Decoder Model

In [33]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
#     self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):

    x = self.embedding(x)


    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [34]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (4, 20847)


## Define the optimizer and the loss function

In [35]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [36]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [37]:
train_loss = tf.keras.metrics.Mean(name='train_loss')


In [38]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [39]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.1070
Epoch 1 Batch 100 Loss 1.4963
Epoch 1 Batch 200 Loss 0.9471
Epoch 1 Batch 300 Loss 0.7991
Epoch 1 Batch 400 Loss 0.6944
Epoch 1 Batch 500 Loss 0.6334
Epoch 1 Batch 600 Loss 0.6683
Epoch 1 Batch 700 Loss 1.1812
Epoch 1 Batch 800 Loss 0.6730
Epoch 1 Batch 900 Loss 0.8666
Epoch 1 Batch 1000 Loss 0.8624
Epoch 1 Batch 1100 Loss 0.8810
Epoch 1 Batch 1200 Loss 0.4791
Epoch 1 Batch 1300 Loss 0.4143
Epoch 1 Batch 1400 Loss 0.3142
Epoch 1 Batch 1500 Loss 1.3091
Epoch 1 Batch 1600 Loss 0.9892
Epoch 1 Batch 1700 Loss 0.8338
Epoch 1 Batch 1800 Loss 1.3493
Epoch 1 Batch 1900 Loss 1.0124
Epoch 1 Batch 2000 Loss 0.5209
Epoch 1 Batch 2100 Loss 1.3138
Epoch 1 Batch 2200 Loss 0.6569
Epoch 1 Batch 2300 Loss 1.1101
Epoch 1 Batch 2400 Loss 0.5306
Epoch 1 Batch 2500 Loss 0.8378
Epoch 1 Batch 2600 Loss 0.6960
Epoch 1 Batch 2700 Loss 1.7259
Epoch 1 Batch 2800 Loss 0.4000
Epoch 1 Batch 2900 Loss 0.4398
Epoch 1 Batch 3000 Loss 0.4738
Epoch 1 Batch 3100 Loss 1.1205
Epoch 1 Batch 3200 L

In [40]:
(876+744)/60

27.0

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [41]:
def evaluate(sentence):


  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)


    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [43]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))



## Restore the latest checkpoint and test

In [44]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [45]:
inp_lang.word_index

{'.': 1,
 ',': 2,
 '<start>': 3,
 '<end>': 4,
 'o': 5,
 'a': 6,
 'de': 7,
 'que': 8,
 'e': 9,
 'n': 10,
 's': 11,
 'um': 12,
 'uma': 13,
 'para': 14,
 'os': 15,
 'se': 16,
 'em': 17,
 'as': 18,
 'do': 19,
 'com': 20,
 'da': 21,
 'm': 22,
 'como': 23,
 'no': 24,
 'mas': 25,
 'por': 26,
 '?': 27,
 'na': 28,
 'mais': 29,
 'eu': 30,
 'es': 31,
 'est': 32,
 'nos': 33,
 'muito': 34,
 'isso': 35,
 'isto': 36,
 'ser': 37,
 'ou': 38,
 'pessoas': 39,
 'me': 40,
 't': 41,
 'dos': 42,
 'ao': 43,
 'quando': 44,
 'fazer': 45,
 'h': 46,
 'ncia': 47,
 'porque': 48,
 'foi': 49,
 'ent': 50,
 'esta': 51,
 'vel': 52,
 'era': 53,
 'das': 54,
 'este': 55,
 'todos': 56,
 'anos': 57,
 'temos': 58,
 'ele': 59,
 'sobre': 60,
 'eles': 61,
 'c': 62,
 'v': 63,
 'ver': 64,
 'tamb': 65,
 'aqui': 66,
 'agora': 67,
 'mundo': 68,
 'tem': 69,
 'ter': 70,
 'minha': 71,
 'mesmo': 72,
 'meu': 73,
 'podem': 74,
 'bem': 75,
 'ria': 76,
 'come': 77,
 'forma': 78,
 'podemos': 79,
 'mos': 80,
 'voc': 81,
 'pode': 82,
 'coisas':

In [46]:
translate("problema que temos que resolver.")

Input: <start> problema que temos que resolver . <end>
Predicted translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 


In [47]:
translate("este é um problema que temos que resolver.")

Input: <start> este um problema que temos que resolver . <end>
Predicted translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 


In [48]:
translate("os meus vizinhos ouviram sobre esta ideia.")

Input: <start> os meus vizinhos ouviram sobre esta ideia . <end>
Predicted translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 


In [49]:
translate("os meus alunos t m problemas")

Input: <start> os meus alunos t m problemas <end>
Predicted translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 


## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs


In [50]:
inp_lang_val, targ_lang_val = create_dataset(val_examples,None,None)

In [51]:
inp_lang_val[1]

'<start> estava sempre preocupado em ser apanhado e enviado de volta . <end>'

In [52]:
targ_lang_val[1]

'<start> i was always worried about being caught and sent back . <end>'

In [57]:
bleuplus = 0
bleu_ = 0

for i in [0,1,5,70,80,101,6,99,87,999,105,33,45,89,96]: #0,1,5
    pt_test = inp_lang_val[i]
    print('Input: {}'.format(pt_test))
#     pt_test = pt_.numpy().decode("utf-8", "ignore")
    result, sentence = evaluate(pt_test)
# #     predicted = translate(pt_test)
    print('Predicted Translation: {}'.format(result))
    en_test = targ_lang_val[i].split(' ', 1)[1].rsplit(' ', 1)[0]
    print('Real translation: {}'.format(en_test))
    bleu_ = bleu.sentence_bleu([en_test.split()], result.split(), weights=(1,0,0,0))
    print("BLEU Score: ", bleu_)
    print("----" *20)

    bleuplus += bleu_

Input: <start> tinham comido peixe com batatas fritas ? <end>
Predicted Translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 
Real translation: did they eat fish and chips ?
BLEU Score:  0.0072992700729927
--------------------------------------------------------------------------------
Input: <start> estava sempre preocupado em ser apanhado e enviado de volta . <end>


C:\Users\user\Anaconda3\envs\tf-gpu-env\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\user\Anaconda3\envs\tf-gpu-env\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\user\Anaconda3\envs\tf-gpu-env\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-g

Predicted Translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 
Real translation: i was always worried about being caught and sent back .
BLEU Score:  0.0072992700729927
--------------------------------------------------------------------------------
Input: <start> e , na minha opini o , uma cura n o s para mim , mas para toda a gente . <end>
Predicted Translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the worl

Predicted Translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and 
Real translation: and without the humor .
BLEU Score:  0.014598540145985401
--------------------------------------------------------------------------------
Input: <start> e estas s o as quest es com que temos de nos preocupar durante os pr ximos anos . <end>
Predicted Translation: and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , and the world , a

In [58]:
bleuplus/15

0.014111922141119223